# Applied Data Science Capstone.

### Week 03 - Segmenting and Clustering Neighborhoods in Toronto.

**Author:** Felipe Demenech Vasconcelos

# Imports and Functions.

In [8]:
# pip install geocoder

In [9]:
import folium
import requests
import geocoder
import pandas as pd

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

# Loading Data.

## Loading data using Pandas.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df_pd = pd.read_html(url, index_col = 0)[0]
df_pd.reset_index(inplace = True)

# Excluding "Not assigned" rows
df_pd = df_pd[df_pd['Borough'] != 'Not assigned']

df_pd.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Loading data using BeautifulSoup.

In [3]:
resp = requests.get(url)
resp.status_code

200

In [4]:
# Creating a BeautifulSoup object
soup = BeautifulSoup(resp.text, 'html.parser')
tb = soup.findAll('table')[0]

In [5]:
df_soup = pd.read_html(str(tb))

# Transforming BeautifulSoup object into a Pandas dataframe
df_soup = pd.DataFrame(df_soup[0])

# Excluding "Not assigned" rows
df_soup = df_soup[df_soup['Borough'] != 'Not assigned']

df_soup.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Data Dimensions

In [6]:
print(f'df_pd dimensions: \n- rows: {df_pd.shape[0]}. \n- columns: {df_pd.shape[1]}.\n')
print(f'df_soup dimensions: \n- rows: {df_soup.shape[0]}. \n- columns: {df_soup.shape[1]}.')

df_pd dimensions: 
- rows: 103. 
- columns: 3.

df_soup dimensions: 
- rows: 103. 
- columns: 3.


# Getting location coordinates

In [11]:
# the geocoder method doesn't worked.

'''
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(df_soup['Postal Code']))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
'''

KeyboardInterrupt: 

In [12]:
# Loading the coordinates csv file.

df_loc = pd.read_csv('C:\\Users\\felip\\CURSOS_ALURA\\IBM Data Science\\Applied DS Capstone\\Coursera_Capstone\\data\\Geospatial_Coordinates.csv')
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
# Concatenating both dataframes.

df = pd.merge(df_soup, df_loc, on = 'Postal Code')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
